# Lab 1: Streaming Lakehouse Fundamentals

## 🎯 **Learning Objectives:**
- Hiểu Streaming Lakehouse architecture là gì
- So sánh Lambda vs Kappa vs Streaming Lakehouse
- Tại sao "End of Lambda Architecture"?
- Unified pipeline concept
- Domain: Stock Trading Data

## 📚 **Key Concepts:**
1. **Streaming Lakehouse**: Unified batch + streaming trên cùng storage
2. **Lambda Architecture**: 2 pipelines riêng (batch + streaming)
3. **Kappa Architecture**: 1 pipeline streaming only
4. **Unified Pipeline**: Batch và streaming dùng chung code
5. **Medallion Architecture**: Bronze → Silver → Gold


## 1. Lambda Architecture - Problems

### Architecture:
```
┌─────────────────┐
│  Batch Layer    │  Hadoop/Spark Batch
│  (Hadoop/Spark) │  - Historical data
└────────┬────────┘  - High accuracy
         │
         ├─────────────────┐
         │                 │
         ▼                 ▼
┌─────────────────┐  ┌─────────────────┐
│ Streaming Layer │  │  Serving Layer  │
│ (Kafka Streams) │  │  (Results)      │
│ - Real-time     │  │                 │
└─────────────────┘  └─────────────────┘
```

### Problems:
- ❌ **2 codebases**: Logic phải viết 2 lần (batch + streaming)
- ❌ **2 systems**: Cần maintain 2 hệ thống song song
- ❌ **Complexity**: Phức tạp, tốn kém
- ❌ **Inconsistency**: Có thể có kết quả khác nhau giữa batch và streaming
- ❌ **High cost**: 2x infrastructure

### Example:
```python
# Batch code
def process_batch(data):
    return data.groupBy("symbol").agg(avg("price"))

# Streaming code (phải viết lại!)
def process_stream(data):
    return data.groupBy("symbol").agg(avg("price"))  # Same logic!
```


## 2. Kappa Architecture - Limitations

### Architecture:
```
┌─────────────┐
│ Kafka Log   │  Event log
└──────┬──────┘
       │
       ▼
┌─────────────────┐
│ Streaming Only  │  Single pipeline
│ (Kafka Streams) │  - Reprocess từ log
└──────┬──────────┘  - Real-time
       │
       ▼
┌─────────────────┐
│   Results       │
└─────────────────┘
```

### Advantages:
- ✅ **1 pipeline**: Đơn giản hơn Lambda
- ✅ **1 codebase**: Chỉ cần maintain 1 code
- ✅ **Real-time**: Phù hợp event-driven

### Limitations:
- ⚠️ **Reprocessing cost**: Phải reprocess toàn bộ data từ log
- ⚠️ **Historical data**: Khó xử lý historical data lớn
- ⚠️ **Storage**: Kafka log không phải storage format tốt cho analytics
- ⚠️ **Query**: Khó query historical data hiệu quả

### Example Problem:
```python
# Cần historical data 1 năm trước?
# → Phải reprocess toàn bộ 1 năm từ Kafka log
# → Tốn kém, chậm
```


## 3. Streaming Lakehouse - Solution ⭐

### Architecture:
```
┌─────────────┐
│   Kafka     │  Real-time events
└──────┬──────┘
       │
       ▼
┌─────────────────────────────────────┐
│   Spark Structured Streaming         │
│   (Unified Processing Logic)         │
│   - Same code for batch & stream     │
└──────┬───────────────────────────────┘
       │
       ▼
┌─────────────────────────────────────┐
│   Apache Iceberg Tables              │
│   ┌─────────┐  ┌─────────┐  ┌──────┐ │
│   │ Bronze  │→ │ Silver  │→ │ Gold │ │
│   │ (Raw)   │  │(Cleaned)│  │(Agg) │ │
│   └─────────┘  └─────────┘  └──────┘ │
└─────────────────────────────────────┘
       │
       ├─────────────────┬──────────────┐
       ▼                 ▼              ▼
┌──────────┐    ┌──────────────┐  ┌─────────┐
│ Real-time│    │ Batch Reports│  │   ML    │
│ Analytics│    │ (Same Data) │  │ Models  │
└──────────┘    └──────────────┘  └─────────┘

Parallel: Batch Processing (Same Code)
┌─────────────┐
│ Batch Files │  Historical data
└──────┬──────┘
       │
       ▼
┌─────────────────────────────────────┐
│   Spark Batch (Same Functions)       │
└──────┬───────────────────────────────┘
       │
       ▼
┌─────────────────────────────────────┐
│   Same Iceberg Tables                │
└─────────────────────────────────────┘
```

### Key Advantages:
- ✅ **1 codebase**: Batch và streaming dùng chung code
- ✅ **1 storage**: Cùng Iceberg tables
- ✅ **Historical data**: Lưu trong Iceberg, không cần reprocess
- ✅ **Unified query**: Cùng query engine cho real-time và historical
- ✅ **ACID transactions**: Iceberg đảm bảo consistency
- ✅ **Time travel**: Query data tại bất kỳ thời điểm nào


In [ ]:
# Comparison: Lambda vs Kappa vs Streaming Lakehouse
print("📊 Comparison: Lambda vs Kappa vs Streaming Lakehouse")
print("=" * 80)

comparison = {
    "Aspect": ["Pipelines", "Codebases", "Historical Data", "Complexity", "Cost", "Query Flexibility"],
    "Lambda": [
        "2 (batch + stream)",
        "2 (duplicate logic)",
        "✅ Good (batch layer)",
        "⚠️ High",
        "⚠️ High (2x infra)",
        "⚠️ Different results"
    ],
    "Kappa": [
        "1 (stream only)",
        "1",
        "❌ Poor (reprocess from log)",
        "✅ Low",
        "✅ Low",
        "⚠️ Limited (stream only)"
    ],
    "Streaming Lakehouse": [
        "1 (unified)",
        "1 (shared code)",
        "✅ Excellent (Iceberg storage)",
        "✅ Low",
        "✅ Low",
        "✅ Excellent (unified query)"
    ]
}

import pandas as pd
df_comparison = pd.DataFrame(comparison)
print("\n" + df_comparison.to_string(index=False))

print("\n💡 Key Insight:")
print("   Streaming Lakehouse = Best of both worlds:")
print("   - Simplicity of Kappa (1 pipeline)")
print("   - Historical data capability of Lambda (Iceberg storage)")
print("   - Plus: Unified code, unified query, ACID transactions")


## 4. Domain: Stock Trading Data

### Use Case: Real-time Stock Market Analytics

**Data Types**:
1. **Trade Events**: Actual stock trades (buy/sell)
2. **Quote Events**: Bid/ask prices
3. **Market Data**: Volume, price movements

### Data Schema

**Stock Trade Event**:
```json
{
  "trade_id": "TRD_001",
  "symbol": "AAPL",
  "price": 175.50,
  "volume": 100,
  "timestamp": "2025-01-15T10:30:00Z",
  "trade_type": "BUY",
  "exchange": "NASDAQ"
}
```

**Stock Quote Event**:
```json
{
  "quote_id": "QTE_001",
  "symbol": "AAPL",
  "bid": 175.45,
  "ask": 175.55,
  "timestamp": "2025-01-15T10:30:00Z",
  "exchange": "NASDAQ"
}
```

### Use Cases:
- **Real-time**: Live trading dashboard, price alerts
- **Historical**: Backtesting strategies, trend analysis
- **ML**: Price prediction, volatility models
- **Analytics**: Volume analysis, market patterns


## 5. Unified Pipeline Concept

### Key Idea: Same Code, Different Triggers

```python
# Shared transformation function (dùng cho cả batch và streaming)
def process_stock_data(df):
    """Process stock trades - works for both batch and streaming"""
    return (df
        .withColumn("total_value", col("price") * col("volume"))
        .groupBy("symbol", window("timestamp", "1 minute"))
        .agg(
            avg("price").alias("avg_price"),
            sum("volume").alias("total_volume"),
            sum("total_value").alias("total_value")
        )
    )

# Streaming: Real-time processing
stream_df = spark.readStream.format("kafka")...
result_stream = process_stock_data(stream_df)
result_stream.writeStream.format("iceberg").save("warehouse.gold.trades")

# Batch: Historical processing (SAME CODE!)
batch_df = spark.read.parquet("historical_trades/")
result_batch = process_stock_data(batch_df)  # Same function!
result_batch.write.format("iceberg").save("warehouse.gold.trades")  # Same table!
```

### Benefits:
- ✅ **Code reuse**: Viết 1 lần, dùng cho cả batch và streaming
- ✅ **Consistency**: Cùng logic → cùng kết quả
- ✅ **Maintainability**: Chỉ cần maintain 1 codebase
- ✅ **Testing**: Test 1 function cho cả 2 use cases


## Summary

### ✅ Key Takeaways:
1. **Lambda Architecture**: 2 pipelines → Phức tạp, tốn kém
2. **Kappa Architecture**: 1 pipeline streaming → Đơn giản nhưng khó xử lý historical
3. **Streaming Lakehouse**: 1 unified pipeline → Đơn giản + xử lý được historical
4. **Unified Code**: Batch và streaming dùng chung transformation logic
5. **Unified Storage**: Cùng Iceberg tables cho cả batch và streaming
6. **Medallion Architecture**: Bronze → Silver → Gold với streaming

### 🎯 Why Streaming Lakehouse?
- **End of Lambda**: Không cần 2 pipelines nữa
- **Better than Kappa**: Có thể xử lý historical data hiệu quả
- **Unified**: 1 codebase, 1 storage, 1 query engine
- **Modern**: Phù hợp với AI/ML, real-time analytics

### 🚀 Next Steps:
- Lab 2: Build Bronze layer (Kafka → Iceberg)
- Lab 3: Build Silver layer (cleaning, dedup)
- Lab 4: Build Gold layer (aggregations)
- Lab 5: Unified batch + streaming
- Lab 6: Query và time travel
